# Think Act Agent

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Optional, Any
from langchain.automaton.think_act_agent import (
    ThinkActAgent,
    generate_memory,
    ThinkActPromptGenerator,
)
from langchain.schema.messages import HumanMessage
from langchain.llms import OpenAI
from langchain.tools import tool, Tool

In [3]:
def get_tools() -> List[Tool]:
    @tool
    def name() -> str:
        """Use to look up the user's name"""
        return "Eugene"

    @tool
    def get_weather(city: str) -> str:
        """Get weather in a specific city."""
        return "42F and sunny"

    @tool
    def add(x: int, y: int) -> int:
        """Use to add two numbers. For example; { "x": 2, "y": 10}"""
        return x + y

    @tool
    def divide(x: float, y: float) -> float:
        """Divide x by y"""
        return x / y

    return list(locals().values())


tools = get_tools()

In [4]:
llm = OpenAI(temperature=0)
agent = ThinkActAgent(llm, tools)

In [5]:
message_log = generate_memory(tools)
question = HumanMessage(content="what is the 2 + 5?")
message_log.add_messages([question])
agent.run(message_log)

In [7]:
print(ThinkActPromptGenerator.from_message_log(message_log).to_string())

Respond to the human as helpfully and accurately as 
possible. You have access to the following tools:
name: name() -> str - Use to look up the user's name
get_weather: get_weather(city: str) -> str - Get weather in a specific city.
add: add(x: int, y: int) -> int - Use to add two numbers. For example; { "x": 2, "y": 10}
divide: divide(x: float, y: float) -> float - Divide x by y

Use a blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or name, get_weather, add, divide

You can only use a single tool at a time.

<action>
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
</action>

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
<action>
$BLOB
</action>

Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
<action>
{
  "action": "Final Answer",
  "action_input": "Final response to human"
